In [ ]:
! pip install streamlit lime pyngrok xgboost shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=fdf3cd5091f093034aa2c02ad53f335afbce8248e101330897a7556130024a09
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [ ]:
%%writefile Home.py
import streamlit as st

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_auc_score,r2_score


st.set_page_config(page_title="Drop_Test_Predict",page_icon='👾',layout='centered')
st.title("AutoML Application")
st.write("""
📤 **Step 1: Upload Your Dataset (CSV) in Drop Tab**

Dataset Format Requirements:

To ensure smooth training and accurate predictions, your CSV file must follow these rules:

✔ The last column must be the Target column

This is the variable the model will learn to predict.

Example: loan_status, species, price, fraud_flag, etc.

✔ The first column should be an ID column

This must contain unique identifiers such as:

- customer_id, applicant_no, serial_number, transaction_id

✔ All other columns are used as features

🤖  **Step 2: Automatic Model Training**

- Once uploaded, the app will:

- Detect if the problem is classification or regression

- Automatically clean and encode your data

Try multiple ML models

Select the best-performing model using industry-standard metrics (AUC for classification, R² for regression)

📝 **Step 3: Enter New Values & Predict**

After training:

- The app generates input boxes for each feature

- Enter values for a new sample

- Click Predict to see the model’s output instantly

🎯 **Purpose of This App**

This platform is designed to:

- Enable non-technical users to build ML models

- Automate training and model selection

- Provide quick predictions without writing code

- Support any dataset with a clear target column""")

Overwriting Home.py


In [ ]:
!mkdir -p pages

In [ ]:
%%writefile pages/1_Upload.py
import streamlit as st
import numpy as np
import zipfile
import os
import kagglehub
from functools import reduce
import pandas as pd
st.set_page_config(page_title="Drop_Test_Predict",page_icon='👾',layout='centered')

if  "df"  in st.session_state:
  st.success("✔️ File already uploaded")
  st.write(f"**Preview: {st.session_state['dataset']}**")
  st.dataframe(st.session_state['df'].head())
else:
  def dataframe(Ufiles):
    try:
      dframe = pd.read_csv(Ufiles)
    except Exception:
      try:
        dframe = pd.read_excel(Ufiles)
      except Exception as e:
        st.error("❌ Could not read file. Unsupported format")
        st.stop()
    return dframe

  def Multi_dataframe(path,Ufiles):
      all_df=[]
      for file_name in Ufiles:
          st.write(file_name)
          if file_name.endswith(("csv", "xls", "xlsx", "xls", "xlsb", "xlsm", "ods")):
            all_df.append(dataframe(os.path.join(path,file_name)))
      df_common = set(all_df[0].columns)
      for dfs in all_df[1:]:
        df_common = df_common.intersection(dfs.columns)
      df_common = list(df_common)
      st.write(f"**🎰Common features: {df_common[0]}**")
      df_merged = reduce(lambda df1, df2: pd.merge(df1, df2, on=df_common, how='outer'), all_df)
      return df_merged

  st.title("📥 Upload Dataset or Enter URL")
  uploaded_file = st.file_uploader("Upload CSV/Excel file or ZIP  file", type=["csv", "xls", "xlsx", "xls", "xlsb", "xlsm", "ods"])
  st.write("***")
  data_url = st.text_input("Enter URL :",placeholder="Paste a direct CSV/Excel or Kaggle Hub link here")
  df=None

  if uploaded_file is not None:
    if uploaded_file.name.split('.')[-1] =="zip":
      path="Data_folder"
      st.write("Multiple datasets detected...")
      with zipfile.ZipFile(uploaded_file, "r") as zip_ref:
        zip_ref.extractall(path)
        files = os.listdir(path)
        with st.spinner("Merging all Dataset"):
          df=Multi_dataframe(path,files)
        st.session_state['df']=df
        st.session_state["dataset"]=uploaded_file.name.split('.')[0]
    else:
      df=dataframe(uploaded_file)
      st.success("File uploaded successfully!")
      st.session_state['df']=df
      st.session_state["dataset"]=uploaded_file.name.split('.')[0]
  elif data_url:
    if data_url.split('.')[-1] not in ("csv", "xls", "xlsx", "xls", "xlsb", "xlsm", "ods"):
      st.write("**Kaggle hub dataset**")
      path = kagglehub.dataset_download(data_url)
      files = os.listdir(path)
      if len(files)>1:
        st.write("**📚Multiple datasets detected**")
        with st.spinner("**📒Merging all Dataset**"):
          df=Multi_dataframe(path,files)
      else:
        df = dataframe(os.path.join(path, files[0]))
    else:
      df=dataframe(data_url)
    st.session_state["dataset"]=data_url.split('/')[-1]
    st.session_state['df']=df
  else:
    pass
  if df is not None:
    st.write(f"**Preview: {st.session_state['dataset']}**")
    st.dataframe(st.session_state['df'].head())


Overwriting pages/1_Upload.py


In [ ]:
%%writefile pages/2_preprocessing.py
import streamlit as st
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import LabelEncoder
st.set_page_config(page_title="Drop_Test_Predict",page_icon='👾',layout='centered')
st.title("🤖 Train Model")
if "df" not in st.session_state:
    st.error("❌ No dataset uploaded. Go to Drop page first.")
    st.stop()

df=st.session_state["df"]
st.dataframe(df.head())
target_col = st.selectbox("🎯 Select Target Column (Prediction Output Column)",options=df.columns)

drop_cols = st.multiselect(" Select Unwanted Columns to Drop (e.g., ID, Serial No, Unique Key)",options=[col for col in df.columns if col != target_col])

if st.button("Apply Selection"):
    df.drop(columns=drop_cols,inplace=True)
    target = df[target_col]
    if target.dtype in ['int64', 'float64']:
        unique_vals = target.nunique()
        if unique_vals <= 20:
            p_type = "classification"
        else:
            p_type = "regression"
    else:
        p_type = "classification"

    st.write(f"Dataset: **{st.session_state['dataset']}**")
    st.session_state['p_type']=p_type
    st.write(f"Detected problem type : **{p_type}**")
    #Preprocessing
    #Handling duplicates
    with st.spinner("🚀 Optimizing your dataset for best model performance..."):
      if df.duplicated().any():
        df=df.drop_duplicates()
      df = df.dropna(subset=[df.columns[-1]])
      X = df.drop(columns=target_col)
      Y = df[target_col]
      for_col=[]
      for_uniq=[]
      for col in  X.select_dtypes(include=["object"]).columns:
        for_uniq.append(X[col].unique().tolist())
        for_col.append(col)
      all=dict(zip(for_col,for_uniq))
      st.session_state["cat_options"]=all
      st.session_state['features']=X
      #Handling missing values
      missing=X.isna().mean().to_dict()
      num_cols = X.select_dtypes(include=["int64", "float64"]).columns
      #cat_cols = X.select_dtypes(include=["object"]).columns
      X_encoders = {}
      for f,m in missing.items():
        if m > 0.5:
          X.drop(columns=f,inplace=True)
        else:
          if f in num_cols:
            X[f].fillna(X[f].median(),inplace=True)
            Q1 = X[f].quantile(0.25)
            Q3 = X[f].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            no_of_outliers=((X[f]<lower_bound)|(X[f]>upper_bound)).sum()
            if (no_of_outliers/len(X[f]))>0.1:
              X[f] = X[f].clip(lower=lower_bound, upper=upper_bound)
          else:
            X[f].fillna(X[f].mode()[0],inplace=True)
            le = LabelEncoder()
            X[f] = le.fit_transform(X[f].astype(str))
            X_encoders[f] = le
      st.session_state['encoders']=X_encoders
      #handling outliers
      if p_type=='regression':
        Q1 = Y.quantile(0.25)
        Q3 = Y.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        Y = Y.clip(lower=lower_bound, upper=upper_bound)
      else:
        target_encoder = LabelEncoder()
        Y = pd.DataFrame(target_encoder.fit_transform(Y.astype(str)), columns=[target_col])
        st.session_state['target_encoder']=target_encoder
      #encoding
      time.sleep(3)
      st.session_state['X']=X
      st.session_state['Y']=Y
      st.success("✨ Preprocessing Complete! Data is now ready for model training 🚀")
      st.write("🧮Features Previews: ")
      st.dataframe(X.head())
      st.write("🔍Prediction Preview: ")
      st.dataframe(Y.head())

Overwriting pages/2_preprocessing.py


In [ ]:
%%writefile pages/3_train.py
import streamlit as st
import pandas as pd
import numpy as np
import time
from sklearn.base import clone
from sklearn.model_selection import train_test_split, cross_val_score,RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler,LabelEncoder,label_binarize
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
# Train-test split
X=st.session_state['X']
Y=st.session_state['Y']
p_type=st.session_state['p_type']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

models = {}

if p_type == "classification":
    models = {
        "Logistic Regression": Pipeline([("scaler", StandardScaler()), ("model", LogisticRegression(max_iter=500))]),
        "SVM": Pipeline([("scaler", StandardScaler()), ("model", SVC(probability=True))]),
        "KNN": Pipeline([("scaler", StandardScaler()), ("model", KNeighborsClassifier())]),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
else:
    models = {
        "Linear Regression": Pipeline([("scaler", StandardScaler()), ("model", LinearRegression())]),
        "SVR": Pipeline([("scaler", StandardScaler()), ("model", SVR())]),
        "KNN Regressor": Pipeline([("scaler", StandardScaler()), ("model", KNeighborsRegressor())]),
        "Random Forest Regressor": RandomForestRegressor(random_state=42),
        "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
        "XGBoost Regressor": XGBRegressor(random_state=42)
    }

results = {}
best_model = None
best_score = 0

with st.spinner(" Training multiple models & selecting the best one..."):
    for name, model in models.items():
        model.fit(X_train, y_train)
        classes = sorted(np.unique(y_test))
        y_pred = model.predict(X_test)

        if p_type == "classification":
          if len(classes) == 2:  # Binary classification
              score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
          else:  # Multi-class classification
              score = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
        else:
            score = r2_score(y_test, y_pred)

        results[name] = score

    time.sleep(2)

results_df = pd.DataFrame({"Model": results.keys(), "Score": results.values()},)
results_df.index=results_df.index+1
results_df = results_df.sort_values(by="Score", ascending=False)

st.success("🎯 Model Training Completed!")
st.write("### 🥇🥈🥉 Top 3 models")
st.dataframe(results_df.head(3))
#Hyperparameter tuning for Classification
param_grid_classification = {
    "Logistic Regression": {
        "model__C": [0.01, 0.1, 1, 10],
        "model__penalty": ["l2"]
    },
    "SVM": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["linear", "rbf"],
        "model__gamma": ["scale", "auto"]
    },
    "KNN": {
        "model__n_neighbors": [3, 5, 7, 11],
        "model__weights": ["uniform", "distance"]
    },
    "Random Forest": {
        "n_estimators": [100, 200, 400],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    },
    "Decision Tree": {
        "max_depth": [None, 10, 20, 30],
        "criterion": ["gini", "entropy"]
    },
    "XGBoost": {
        "n_estimators": [200, 400, 600,800],
        "learning_rate": [0.01, 0.05, 0.1],
        "max_depth": [3, 5, 7,9],
        "subsample": [0.7, 0.8, 1.0]
    }
}
#Hyper parameter tuning for regression
param_grid_regression = {
    "Linear Regression": {},  # No tuning needed
    "SVR": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["rbf", "linear"],
        "model__gamma": ["scale", "auto"]
    },
    "KNN Regressor": {
        "model__n_neighbors": [3, 5, 7, 11],
        "model__weights": ["uniform", "distance"]
    },
    "Random Forest Regressor": {
        "n_estimators": [100, 400, 600,800],
        "max_depth": [None, 10, 20, 30,50],
        "min_samples_split": [2, 5, 7,10]
    },
    "Decision Tree Regressor": {
        "max_depth": [None, 10, 20, 30],
        "criterion": ["squared_error", "absolute_error"]
    },
    "XGBoost Regressor": {
        "n_estimators": [200, 400, 600,800],
        "learning_rate": [0.01, 0.05, 0.1],
        "max_depth": [3,5,7,9],
        "subsample": [0.7, 0.8, 1.0]
    }
}

# Pick top 3 models from baseline,to perform hyperparameter tuning
top2 = results_df.head(3)["Model"].tolist()

if p_type == "classification":
    param_grid = param_grid_classification
    scoring_metric = "roc_auc"
else:
    param_grid = param_grid_regression
    scoring_metric = "r2"
best_tuned_model_name = None
best_tuned_model = None
best_tuned_score = 0

st.write("### 🔧 Hyperparameter tuning in progress for top 3 models...")

for model_name in top2:
    st.write(f"⏳ Tuning: **{model_name}** ...")
    model = clone(models[model_name])

    if param_grid[model_name] == {}:   # No tuning required
      tuned_model = clone(model)      # make a fresh copy
      tuned_model.fit(X_train, y_train)  # train the model

    else:
        rs = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grid[model_name],
            n_iter=10,
            scoring=scoring_metric,
            cv=5,
            random_state=42,
            n_jobs=-1,
            verbose=0
        )
        rs.fit(X_train, y_train)
        tuned_model = rs.best_estimator_

    y_pred = tuned_model.predict(X_test)
    if p_type == "classification":
      classes = sorted(np.unique(y_test))
      if len(classes) == 2:
        score = roc_auc_score(y_test,tuned_model.predict_proba(X_test)[:, 1])
      else:
        score = roc_auc_score(y_test, tuned_model.predict_proba(X_test), multi_class='ovr')
    else:
      score = r2_score(y_test, y_pred)

    if score > best_tuned_score:
        best_tuned_score = score
        best_tuned_model = tuned_model
        best_tuned_model_name = model_name

st.success("🏆 Hyperparameter tuning complete!")
st.write(f"### Best Tuned Model: **{best_tuned_model_name}**")
st.write(f"### Best Tuned Score: **{best_tuned_score:.4f}**")
st.session_state["best_model"] = best_tuned_model

Overwriting pages/3_train.py


In [ ]:
%%writefile pages/4_predict.py
import streamlit as st
import pandas as pd
import numpy as np
import os
st.set_page_config(page_title="Predict", page_icon="🎯", layout="centered")
st.title("🔮 Make Prediction")

# Check if model exists
if "best_model" not in st.session_state or "X" not in st.session_state:
    st.error("❌ Model not trained yet. Please go to Train page first.")
    st.stop()

model = st.session_state["best_model"]
X_train_cols = st.session_state["X"].columns   # processed columns
p_type = st.session_state["p_type"]
encoders = st.session_state.get("encoders", {})
target_encoder = st.session_state.get("target_encoder", None)
cat_values = st.session_state.get("cat_options", {})

st.write("### 📝 Enter values for prediction:")

# Create input controls dynamically
input_values = {}
for col in X_train_cols:
    if col in encoders:  # categorical
        options = cat_values[col] if col in cat_values else list(encoders[col].classes_)
        input_values[col] = st.selectbox(f"{col}", options=options)
    else:
        input_values[col] = st.number_input(f"{col}", value=float(st.session_state["X"][col].median()))

# Convert input to dataframe
input_df = pd.DataFrame([input_values])
st.session_state['input_df']=input_df
# Encode categorical values
for col in input_df.columns:
    if col in encoders:
        try:
            input_df[col] = encoders[col].transform(input_df[col].astype(str))
        except:
            st.warning(f"⚠ Unknown category in '{col}'. Using default value.")
            input_df[col] = encoders[col].transform([encoders[col].classes_[0]])

# Prediction
if st.button("Predict"):
  try:
    prediction = model.predict(input_df)[0]
    st.session_state['prediction'] = prediction  # store prediction
    st.session_state['last_input'] = input_df
    if p_type == "classification":
        proba = model.predict_proba(input_df)[0]
        st.session_state['proba']=proba

        if target_encoder:
            prediction_label = target_encoder.inverse_transform([int(prediction)])[0]
        else:
            prediction_label = prediction

        st.success(f"🎯 Predicted Class: **{prediction_label}**")
    else:  # Regression
        st.success(f"📌 Predicted Value: **{prediction:.4f}**")

  except Exception as e:
        st.error(f"❌ Prediction failed: {e}")
# ---- PRINT BUTTON ----
if st.button("Print / Save Result"):
    if "prediction" not in st.session_state:
        st.error("⚠ Please make a prediction first.")
    else:
        save_row = st.session_state['last_input'].copy()
        save_row["prediction"] = st.session_state['prediction']

        if p_type == "classification":
            save_row["probability"] = float(st.session_state["proba"].max())

        save_file = "prediction_history.csv"

        # Write/append to file
        if not os.path.exists(save_file):
            save_row.to_csv(save_file, mode='w', header=True, index=False)
        else:
            save_row.to_csv(save_file, mode='a', header=False, index=False)

        st.success("📁 Prediction saved successfully!")

        # Add download button
        df_saved = pd.read_csv(save_file)
        st.download_button(
            label="📥 Download Prediction History",
            data=df_saved.to_csv(index=False),
            file_name="prediction_history.csv",
            mime="text/csv"
        )



Overwriting pages/4_predict.py


In [ ]:
%%writefile pages/5_Shap.py
import streamlit as st
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer

st.write("## 🔍 Model Explainability")
model = st.session_state["best_model"]
input_df = st.session_state["input_df"]
X_train = st.session_state["X"]
p_type = st.session_state["p_type"]
prediction=st.session_state['prediction']
try:
    # Attempt SHAP TreeExplainer
    st.info("🪵 Trying SHAP TreeExplainer...")
    shap.initjs()
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(input_df)

    # Classification / regression handling
    if p_type == "classification":
        pred_class = int(prediction)
        if isinstance(shap_values, list):  # multi-class
            shap_vals = shap_values[pred_class][0]
            base_val = explainer.expected_value[pred_class]
        else:  # binary
            shap_vals = shap_values[0]
            base_val = explainer.expected_value
    else:
        shap_vals = shap_values[0]
        base_val = explainer.expected_value

    explanation = shap.Explanation(
        values=shap_vals,
        base_values=base_val,
        data=input_df.iloc[0].values,
        feature_names=input_df.columns.tolist()
    )

    st.success("✔ SHAP explanation generated successfully!")

    # Waterfall plot
    st.write("### ⬇ SHAP Waterfall Plot")
    fig, ax = plt.subplots(figsize=(10, 7))
    shap.plots.waterfall(explanation, max_display=10, show=False)
    st.pyplot(fig)

    # Summary bar plot
    st.write("### 📊 SHAP Feature Importance Ranking")
    fig2, ax2 = plt.subplots(figsize=(8, 6))
    shap.plots.bar(explanation, show=False)
    st.pyplot(fig2)

except Exception as e:
    # If SHAP fails → fallback to LIME
    st.warning(f"⚠ SHAP not available for this model. Switching to LIME automatically.\nError: {e}")

from lime.lime_tabular import LimeTabularExplainer

# Build lime explainer
lime_explainer = LimeTabularExplainer(
    X_train.values,
    feature_names=X_train.columns.tolist(),
    class_names=np.unique(st.session_state["Y"]),
    discretize_continuous=True,
    verbose=True
)

if p_type == "classification":
    pred_class_idx = int(prediction)         # numeric class index
    class_names = np.unique(st.session_state["Y"])
    target_encoder=st.session_state['target_encoder']
    # If encoding exists, reverse translate to actual label
    if target_encoder:
        pred_class_label = target_encoder.inverse_transform([pred_class_idx])[0]
    else:
        pred_class_label = class_names[pred_class_idx]

    # Generate LIME explanation for predicted class
    lime_exp = lime_explainer.explain_instance(
        input_df.values[0],
        model.predict_proba,
        num_features=10,
        top_labels=1
    )

    st.success(f"🎯 LIME explanation for class: **{pred_class_label}**")

    fig3 = lime_exp.as_pyplot_figure(label=pred_class_idx)
    st.pyplot(fig3)

else:  # Regression
    lime_explainer = LimeTabularExplainer(
        X_train.values,
        feature_names=X_train.columns.tolist(),
        discretize_continuous=True,
        verbose=True
    )
    lime_exp = lime_explainer.explain_instance(
        input_df.values[0],
        model.predict,
        num_features=10
    )

    st.write("### 💡 LIME Explanation (Regression)")
    fig3 = lime_exp.as_pyplot_figure()
    st.pyplot(fig3)


Overwriting pages/5_Shap.py


In [ ]:
from pyngrok import ngrok
!ngrok config add-authtoken 35K0ERpK7SgOnSLLKN1IF9ov355_jhsFikHcuWL9dPugML5i

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
# Kill any previous tunnels
ngrok.kill()

# Start Streamlit in the background on port 8051
get_ipython().system_raw('streamlit run Home.py --server.port 8502 &')

public_url = ngrok.connect(8502)
print("Click the public URL to open your app 👇")
print(public_url)

Click the public URL to open your app 👇
NgrokTunnel: "https://shella-unmilitary-laically.ngrok-free.dev" -> "http://localhost:8502"
